In [2]:
import pandas as pd 
import numpy as np
import quandl
import datetime
import math



In [3]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import median_absolute_error

from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score

# https://scikit-learn.org/stable/modules/model_evaluation.html Score pour regression !!!

In [57]:
#Recap fait que les Errors et donc pas des score (les erreur le plus bas le mieux les score le plus haut le mieux)

price_score = pd.DataFrame(index =["AAE","MSE","MSLE","MAE"],columns=['Kalman', 'ARIMA', 'LSTM', 'RF', 'SCM']).fillna(0)
price_score

,Kalman,ARIMA,LSTM,RF,SCM
AAE,0,0,0,0,0
MSE,0,0,0,0,0
MSLE,0,0,0,0,0
MAE,0,0,0,0,0


In [58]:
#Data=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Data_total.csv")
Data=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Data_total2.csv")



# LSTM

In [5]:
df_cie=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Liste_entreprise.csv")
#df_cie=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\logo_all.csv")

df_cie=df_cie.iloc[:, 1:]

logo=df_cie['logo'].tolist()
logo_pred=df_cie['logo_pred'].tolist()

df_cie.head(2)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 39: invalid continuation byte

##               LSTM Price 

In [60]:
Lstm_pred=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\LSTM_All_and_Pred.csv")
Lstm_pred=Lstm_pred.set_index('Date')
Lstm_pred

,AAPL,ABT,ACN,ATVI,ADBE,AMD,GOOGL,GOOG,AMZN,AAL,...,ADBE_Pred,AMD_Pred,GOOGL_Pred,GOOG_Pred,AMZN_Pred,AAL_Pred,BLK_Pred,CBS_Pred,MSFT_Pred,FB_Pred
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-06,100.70,42.56,102.16,36.79,91.02,2.505,759.33,743.62,632.65,41.23,...,95.311543,2.332221,821.821015,798.727789,718.765578,42.261554,332.260116,42.457680,56.186822,110.647089
2016-01-07,96.45,41.54,99.16,36.27,89.11,2.275,741.00,726.39,607.94,40.45,...,94.761174,2.255718,815.535459,791.603724,707.476542,41.473617,319.978282,42.743977,55.705085,108.816903
2016-01-08,96.96,40.67,98.20,35.71,87.85,2.140,730.91,714.47,607.05,40.37,...,91.219783,2.374203,763.569085,742.920279,649.960585,42.040831,333.664731,46.667450,54.257651,102.499933


In [62]:
#Normalisation avec Pred en % du True

Lstm_pred_n=pd.DataFrame()
for l in logo:
    l2=l+'_Pred'
    Lstm_pred_n[l]=Lstm_pred[l]/Lstm_pred[l]
    Lstm_pred_n[l2]=Lstm_pred[l2]/Lstm_pred[l]
    Lstm_pred_n[l+'_Ecart']=abs(Lstm_pred_n[l]-Lstm_pred_n[l2])
    Lstm_pred_n[l+'_Ecart_Carré']=(Lstm_pred_n[l]-Lstm_pred_n[l2])**2



In [63]:
Lstm_pred_n[['AAPL','AAPL_Pred']]

,AAPL,AAPL_Pred
Date,,
2016-01-06,1.0,1.055950
2016-01-07,1.0,1.091931
2016-01-08,1.0,1.121563


In [64]:
##Toutes les valeurs entres 0 et 1
#
#Lstm_pred_n2=pd.DataFrame()
#for l in logo:
#    l2=l+'_Pred'
#    maxi=max(Lstm_pred[l].max(),Lstm_pred[l2].max())
#    Lstm_pred_n2[l]=Lstm_pred[l]/maxi
#    Lstm_pred_n2[l2]=Lstm_pred[l2]/maxi
#    Lstm_pred_n2[l+'_Ecart']=abs(Lstm_pred_n2[l]-Lstm_pred_n2[l2]) #Les % sont pas vraiment respecté ???
#    Lstm_pred_n2[l+'_Ecart_Carré']=(Lstm_pred_n2[l]-Lstm_pred_n[l2])**2
#    
#LL=Lstm_pred_n2.reset_index().drop(columns='Date')
#

####  Méthode en prenant la moyenne des scores par entreprise (que 3 valeurs par enterprise donc semble pas ouf)

In [65]:
Lpn=Lstm_pred_n

df_score_lstm=pd.DataFrame(columns=['Companie','Mean AE','Mean SE','Mean LSE','Median AE'])

cie=[]
AAE=[] #A > Average à la place de mean
MSE=[]
MLSE=[]
MAE=[] #M > Median

for l in logo:
    l2=l+'_Pred'
    cie.append(l)
    AAE.append(mean_absolute_error(Lpn[l],Lpn[l2]))
    MSE.append(mean_squared_error(Lpn[l],Lpn[l2]))
    MLSE.append(mean_squared_log_error(Lpn[l],Lpn[l2]))
    MAE.append(median_absolute_error(Lpn[l],Lpn[l2]))
    
df_score_lstm['Companie']=cie
df_score_lstm['Mean AE']=AAE
df_score_lstm['Mean SE']=MSE
df_score_lstm['Mean LSE']=MLSE
df_score_lstm['Median AE']=MAE


In [66]:
df_score_lstm.head(3)

,Companie,Mean AE,Mean SE,Mean LSE,Median AE
0,AAPL,0.089815,0.008786,0.002088,0.091931
1,ABT,0.058231,0.004734,0.001136,0.081144
2,ACN,0.070186,0.004992,0.001204,0.067424


In [67]:
MSAAE=df_score_lstm['Mean AE'].mean()
MSMSE=df_score_lstm['Mean SE'].mean()
MSMLSE=df_score_lstm['Mean LSE'].mean()
MSMAE=df_score_lstm['Median AE'].mean()

In [68]:
Mean_Score_lstm= [MSAAE, MSMSE, MSMLSE, MSMAE]
Mean_Score_lstm

[0.06584681806453209,
 0.0056371919173865545,
 0.0013490339895733936,
 0.06845075962513186]

####  Méthode en prenant toutes les valeurs et non la moyenne des scores

####  Toutes les transfo sur la moyenne des 3 valeurs

In [69]:
#RAPPEL : Lpn=Lstm_pred_n

trues=[]
predictions=[]

for l in logo_pred:
    trues.append(1)
    predictions.append(Lpn[l].mean())

SAAE = mean_absolute_error(trues,predictions)
SMSE = mean_squared_error(trues,predictions)
SMLSE= mean_squared_log_error(trues,predictions)
SMAE = median_absolute_error(trues,predictions)

Score_lstm_mean= [SAAE, SMSE, SMLSE, SMAE]
Score_lstm_mean

[0.06194612936191692,
 0.004602832732344061,
 0.0011059869718618722,
 0.06306660284897936]

####  Toutes les transfo toutes les valeurs

In [70]:
trues2=[]
predictions2=[]

for l in logo_pred:
    for k in Lpn[l]:
        trues2.append(1)
        predictions2.append(k)
        
TSAAE = mean_absolute_error(trues,predictions)
TSMSE = mean_squared_error(trues,predictions)
TSMLSE= mean_squared_log_error(trues,predictions)
TSMAE = median_absolute_error(trues,predictions)

Score_lstm= [TSAAE, TSMSE, TSMLSE, TSMAE]
Score_lstm  

[0.06194612936191692,
 0.004602832732344061,
 0.0011059869718618722,
 0.06306660284897936]

In [71]:
Mean_Score_lstm  ,"    ",  Score_lstm_mean, "    " ,   Score_lstm  

([0.06584681806453209,
  0.0056371919173865545,
  0.0013490339895733936,
  0.06845075962513186],
 '    ',
 [0.06194612936191692,
  0.004602832732344061,
  0.0011059869718618722,
  0.06306660284897936],
 '    ',
 [0.06194612936191692,
  0.004602832732344061,
  0.0011059869718618722,
  0.06306660284897936])

In [72]:
price_score['LSTM']=Score_lstm
price_score*100

,Kalman,ARIMA,LSTM,RF,SCM
AAE,0,0,6.194613,0,0
MSE,0,0,0.460283,0,0
MSLE,0,0,0.110599,0,0
MAE,0,0,6.306660,0,0


# KALMAN

In [73]:
Kalman_pred=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\Kalman_All_and_Pred.csv")
Kalman_pred=Kalman_pred.set_index('Date')
Kalman_pred

,AAPL,AAPL_Pred,ABT,ABT_Pred,ACN,ACN_Pred,ATVI,ATVI_Pred,ADBE,ADBE_Pred,...,AAL,AAL_Pred,BLK,BLK_Pred,CBS,CBS_Pred,MSFT,MSFT_Pred,FB,FB_Pred
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-06,100.70,108.346263,42.56,44.451808,102.16,104.520056,36.79,38.195728,91.02,92.811875,...,41.23,42.448046,330.16,337.739216,46.13,47.195539,54.05,55.135631,102.97,104.664284
2016-01-07,96.45,107.214632,41.54,44.174823,99.16,104.010181,36.27,38.012544,89.11,92.459735,...,40.45,42.257982,315.73,335.645591,45.34,47.019031,52.17,54.853526,97.92,104.022735
2016-01-08,96.96,106.239161,40.67,43.841427,98.20,103.457488,35.71,37.793514,87.85,92.021234,...,40.37,42.078388,307.78,332.994879,46.46,46.965853,52.33,54.613475,97.33,103.386089


In [74]:
#Normalisation avec Pred en % du True

Kalman_pred_n=pd.DataFrame()
for l in logo:
    l2=l+'_Pred'
    Kalman_pred_n[l]=Kalman_pred[l]/Kalman_pred[l]
    Kalman_pred_n[l2]=Kalman_pred[l2]/Kalman_pred[l]
   # Kalman_pred_n[l+'_Ecart']=abs(Kalman_pred[l]-Kalman_pred[l2])
   # Kalman_pred_n[l+'_Ecart_Carré']=(Kalman_pred[l]-Kalman_pred[l2])**2

In [75]:
Kalman_pred_n

,AAPL,AAPL_Pred,ABT,ABT_Pred,ACN,ACN_Pred,ATVI,ATVI_Pred,ADBE,ADBE_Pred,...,AAL,AAL_Pred,BLK,BLK_Pred,CBS,CBS_Pred,MSFT,MSFT_Pred,FB,FB_Pred
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-06,1.0,1.075931,1.0,1.044450,1.0,1.023102,1.0,1.038210,1.0,1.019687,...,1.0,1.029543,1.0,1.022956,1.0,1.023099,1.0,1.020086,1.0,1.016454
2016-01-07,1.0,1.111608,1.0,1.063429,1.0,1.048913,1.0,1.048044,1.0,1.037591,...,1.0,1.044697,1.0,1.063078,1.0,1.037032,1.0,1.051438,1.0,1.062324
2016-01-08,1.0,1.095701,1.0,1.077980,1.0,1.053539,1.0,1.058345,1.0,1.047481,...,1.0,1.042318,1.0,1.081925,1.0,1.010888,1.0,1.043636,1.0,1.062222


In [76]:
Kpn=Kalman_pred_n

df_score_kalman=pd.DataFrame(columns=['Companie','Mean AE','Mean SE','Mean LSE','Median AE'])


trues=[]
predictions=[]

for l in logo_pred:
    for k in Kpn[l]:
        trues.append(1)
        predictions.append(k)
        
TSAAE = mean_absolute_error(trues,predictions)
TSMSE = mean_squared_error(trues,predictions)
TSMLSE= mean_squared_log_error(trues,predictions)
TSMAE = median_absolute_error(trues,predictions)

Score_kalman= [TSAAE, TSMSE, TSMLSE, TSMAE]
Score_kalman  

[0.05308333151957322,
 0.004105760432481327,
 0.0009736831012384756,
 0.04457355467622548]

In [77]:
price_score['Kalman']=Score_kalman
price_score*100

,Kalman,ARIMA,LSTM,RF,SCM
AAE,5.308333,0,6.194613,0,0
MSE,0.410576,0,0.460283,0,0
MSLE,0.097368,0,0.110599,0,0
MAE,4.457355,0,6.306660,0,0


# ARIMA

In [78]:
ARIMA_pred=pd.read_csv(r"C:\Users\Geoffroy\Desktop\IT\Projet Supélec\Data_quandl\ARIMA_All_and_Pred.csv")
ARIMA_pred=ARIMA_pred.set_index('Date')
ARIMA_pred

,AAPL,AAPL_ARIMA,ABT,ABT_ARIMA,ACN,ACN_ARIMA,ATVI,ATVI_ARIMA,ADBE,ADBE_ARIMA,...,AAL,AAL_ARIMA,BLK,BLK_ARIMA,CBS,CBS_ARIMA,MSFT,MSFT_ARIMA,FB,FB_ARIMA
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-06,100.70,102.849469,42.56,43.104821,102.16,103.049687,36.79,37.424541,91.02,92.701094,...,41.23,40.372553,330.16,334.254492,46.13,46.143625,54.05,55.152921,102.97,103.474411
2016-01-07,96.45,100.998130,41.54,42.808513,99.16,102.576268,36.27,37.114829,89.11,91.367640,...,40.45,41.343079,315.73,329.934949,45.34,46.057104,52.17,54.058193,97.92,103.575202
2016-01-08,96.96,96.705969,40.67,41.577668,98.20,99.740070,35.71,36.416567,87.85,89.429808,...,40.37,40.291241,307.78,315.656298,46.46,45.291148,52.33,52.084540,97.33,97.544629


In [79]:
#Normalisation avec Pred en % du True

ARIMA_pred_n=pd.DataFrame()

for l in logo:
    try:
        l2=l+'_ARIMA'
        ARIMA_pred_n[l]=ARIMA_pred[l]/ARIMA_pred[l]
        ARIMA_pred_n[l2]=ARIMA_pred[l2]/ARIMA_pred[l]
       # ARIMA_pred_n[l+'_Ecart']=abs(ARIMA_pred[l]-ARIMA_pred[l2])
       # ARIMA_pred_n[l+'_Ecart_Carré']=(ARIMA_pred[l]-ARIMA_pred[l2])**2
    except:
        pass    


In [80]:
ARIMA_pred_n

,AAPL,AAPL_ARIMA,ABT,ABT_ARIMA,ACN,ACN_ARIMA,ATVI,ATVI_ARIMA,ADBE,ADBE_ARIMA,...,AAL,AAL_ARIMA,BLK,BLK_ARIMA,CBS,CBS_ARIMA,MSFT,MSFT_ARIMA,FB,FB_ARIMA
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-06,1.0,1.021345,1.0,1.012801,1.0,1.008709,1.0,1.017248,1.0,1.018470,...,1.0,0.979203,1.0,1.012402,1.0,1.000295,1.0,1.020406,1.0,1.004899
2016-01-07,1.0,1.047155,1.0,1.030537,1.0,1.034452,1.0,1.023293,1.0,1.025335,...,1.0,1.022079,1.0,1.044991,1.0,1.015816,1.0,1.036193,1.0,1.057753
2016-01-08,1.0,0.997380,1.0,1.022318,1.0,1.015683,1.0,1.019786,1.0,1.017983,...,1.0,0.998049,1.0,1.025591,1.0,0.974842,1.0,0.995309,1.0,1.002205


In [81]:
Apn=ARIMA_pred_n

df_score_kalman=pd.DataFrame(columns=['Companie','Mean AE','Mean SE','Mean LSE','Median AE'])


trues=[]
predictions=[]

for p in logo:
    try:
        l=p+'_ARIMA'
        for k in Apn[l]:
            trues.append(1)
            predictions.append(k)
    except:
        pass
        
TSAAE = mean_absolute_error(trues,predictions)
TSMSE = mean_squared_error(trues,predictions)
TSMLSE= mean_squared_log_error(trues,predictions)
TSMAE = median_absolute_error(trues,predictions)

Score_ARIMA= [TSAAE, TSMSE, TSMLSE, TSMAE]
Score_ARIMA  

[0.0199111963524565,
 0.0005937677789747236,
 0.0001459038195184978,
 0.018469500061139232]

In [82]:
price_score['ARIMA']=Score_ARIMA
price_score*100

,Kalman,ARIMA,LSTM,RF,SCM
AAE,5.308333,1.991120,6.194613,0,0
MSE,0.410576,0.059377,0.460283,0,0
MSLE,0.097368,0.014590,0.110599,0,0
MAE,4.457355,1.846950,6.306660,0,0
